<a href="https://colab.research.google.com/github/Geilson-Araujo/Desafios_NLP_MBA_PUC_Rio/blob/main/puc_desafio_02_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U --quiet spacy
!python -m spacy validate

2021-04-07 06:56:39.993953: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.0.5) =================
ℹ spaCy installation: /usr/local/lib/python3.7/dist-packages/spacy

No pipeline packages found in your current environment.



In [2]:
import pandas as pd
import spacy

print(spacy.__version__)

3.0.5


In [3]:
!python -m spacy debug data config.cfg

2021-04-07 06:56:45.681292: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

============================ Data file validation ============================
✔ Corpus is loadable
✔ Pipeline can be initialized with data

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, ner
5823 training docs
4650 evaluation docs
✔ No overlap between training and evaluation data

============================== Vocab & Vectors ==============================
ℹ 1367193 total word(s) in the data (57662 unique)
ℹ No word vectors present in the package

========================== Named Entity Recognition ==========================
ℹ 0 new label(s), 3 existing label(s)
0 missing value(s) (tokens with '-' label)
⚠ 231 entity span(s) with punctuation
✔ Good amount of examples for all labels
✔ Examples without occurrences available for all labels
✔ No entities consisting of or 

In [4]:
%%time
!python -m spacy train config.cfg --gpu-id 0 --output ./

2021-04-07 06:58:20.613776: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-04-07 06:58:22,533] [INFO] Set up nlp object from config
[2021-04-07 06:58:22,542] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-04-07 06:58:22,546] [INFO] Created vocabulary
[2021-04-07 06:58:22,546] [INFO] Finished initializing nlp object
[2021-04-07 06:58:39,255] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    101.36    0.00    0.00    0.00    0.00
  0     200       1554.23   4982.49    0.00    0.00    0.00    0.00

In [5]:
# Carrega base de dados
corona_dataset = pd.read_csv('./corona_dataset.csv')

# Remove colunas sem desnecessárias
corona_dataset.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis='columns',
                    inplace=True)

# Remove dados faltantes
corona_dataset.dropna(inplace=True)

# Exibe as linhas inicias da base de dados
corona_dataset.head()

,date,title,category,body,source
0,2020-03-19,Nets' Access to COVID-19 Tests Questioned,Pro Basketball,New York Mayor Bill de Blasio is among those q...,By Reuters
1,March 14,Ireland Announces Second Death From COVID-19,Europe,A second patient has died of the COVID-19 viru...,By Reuters
2,March 6,England to Register COVID-19 as 'Notifiable Di...,Europe,"England will formally register COVID-19, a dis...",By Reuters
3,March 18,"Lakers Reportedly Under Quarantine, Will Test ...",Pro Basketball,Members of the Los Angeles Lakers are under qu...,By Reuters
4,March 17,Factbox: COVID-19 and the New Coronavirus-Fact...,Europe,Social media is awash with myths about how peo...,By Reuters


In [10]:
corpus = ' '.join(corona_dataset['body'])

# Carrega o modelo customizado e cria um objeto nlp
spacy.prefer_gpu()
nlp = spacy.load('./model-best')

# Processa o texto
doc = nlp(corpus)

In [11]:
for token in doc.ents:
    print(f'{token.text:<20} {token.label_:>10}')

President             CHEMICALS
Saturday              CHEMICALS
staring               CHEMICALS
ibuprofen             CHEMICALS
ATP                   CHEMICALS
WTA                   CHEMICALS
President             CHEMICALS
Co                    CHEMICALS
Commission            CHEMICALS
FTC                   CHEMICALS
ibuprofen             CHEMICALS
President             CHEMICALS
President             CHEMICALS
COVID-19              CHEMICALS
Saturday              CHEMICALS
Co                    CHEMICALS
nitrogen              CHEMICALS
President             CHEMICALS
headquarters          CHEMICALS
President             CHEMICALS
Saturday              CHEMICALS
Eli                   CHEMICALS
Co                    CHEMICALS
President             CHEMICALS
President             CHEMICALS
President             CHEMICALS
Golden                CHEMICALS
Saturday              CHEMICALS
Oscar                 CHEMICALS
Dubai                 CHEMICALS
Saturday              CHEMICALS
Presiden

In [12]:
# entities_cat_1 = {"GGP":"#F9E79F", "SO":"#F7DC6F", "TAXON":"#F4D03F", "CHEBI":"#FAD7A0", "GO":"#F8C471", "CL":"#F5B041"}
# entities_cat_2 = {"DNA":"#82E0AA", "CELL_TYPE":"#AED6F1", "CELL_LINE":"#E8DAEF", "RNA":"#82E0AA", "PROTEIN":"#82E0AA"}
# entities_cat_3 = {"DISEASE":"#D7BDE2", "CHEMICAL":"#D2B4DE"}
# entities_cat_4 = {"CANCER":"#ABEBC6", "ORGAN":"#82E0AA", "TISSUE":"#A9DFBF", "ORGANISM":"#A2D9CE", "CELL":"#76D7C4", \
#                   "AMINO_ACID":"#85C1E9", "GENE_OR_GENE_PRODUCT":"#AED6F1", "SIMPLE_CHEMICAL":"#76D7C4", "ANATOMICAL_SYSTEM":"#82E0AA", \
#                   "IMMATERIAL_ANATOMICAL_ENTITY":"#A2D9CE", "MULTI-TISSUE_STRUCTURE":"#85C1E9", "DEVELOPING_ANATOMICAL_STRUCTURE":"#A9DFBF", \
#                   "ORGANISM_SUBDIVISION":"#58D68D", "CELLULAR_COMPONENT":"#7FB3D5"}

options = {'ents':["CHEMICALS", 'DRUG-PROTEIN', "DISEASE"],
           'colors':{"CHEMICALS": "#D2B4DE", 'DRUG-PROTEIN': "#82E0AA", 'DISEASE':"#D7BDE2"}}


spacy.displacy.render(doc[:500], style="ent", jupyter=True, options=options)